In [1]:
from pydantic import BaseModel, Field


class BaseConnexxRequest(BaseModel):
    name: str = Field(
        description="用于大模型tool call的请求名称，必须符合snake case",
        examples=["insert_sku"],
        exclude=True,
    )
    description: str = Field(description="用于大模型检索及理解相应请求功能的介绍", exclude=True)

In [2]:
from datetime import date

from pydantic.json_schema import SkipJsonSchema

In [3]:
class CreateSkuRequest(BaseConnexxRequest):
    name: SkipJsonSchema[str] = "insert_sku"
    description: SkipJsonSchema[str] = "Insert sku"

    sku_code: str = Field(
        ...,
        max_length=30,
        description="Product code, item code, or SKU identifier",
        examples=["10988222"],
    )
    sku_name: str = Field(
        ...,
        max_length=150,
        description="Product name or description",
        examples=["HaYa Group YunNan BaiYao 10001"],
    )
    short_name: str | None = Field(
        None,
        max_length=100,
        description="Short description or alias for the product",
        examples=["Yunnan BaiYao"],
    )
    sku_spec: str | None = Field(
        None, max_length=80, description="Product specifications", examples=["500mg"]
    )
    barcode: str | None = Field(
        None,
        max_length=30,
        description="International barcode (EAN/UPC code). "
        "Separate by English commas if have more than one.",
        examples=["6920282910222"],
    )
    external_code: str | None = Field(
        None,
        max_length=30,
        description="Other unique identifiers for the product",
        examples=["K9202001"],
    )
    type: str | None = Field(None, max_length=30, description="Product type.")
    batch_no: str | None = Field(
        None, max_length=50, description="Product batch number", examples=["23071802"]
    )
    quantity: int | None = Field(
        None, description="Required quantity of the product", examples=[200]
    )
    unit: str = Field(..., max_length=10, description="Sales unit of measurement", examples=["PC"])

    # Fields related to product lifecycle management
    production_date: date | None = Field(
        None, description="Production date (yyyy-MM-dd)", examples=["2023-07-18"]
    )
    expiration_date: date | None = Field(
        None, description="Expiration date (yyyy-MM-dd)", examples=["2025-07-18"]
    )

    shelf_life_duration: int | None = Field(
        None,
        description="Shelf life in hours (rounded to nearest whole hour). If not provided, "
        "default value will be calculated by production_date and expiration_date.",
        examples=[2400],
    )
    expiration_alert_days: int | None = Field(
        None, description="Days before expiration to trigger alert", examples=[360]
    )
    shelf_life_lockup_days: int | None = Field(
        None, description="Number of days to lock product after expiration", examples=[20]
    )
    shelf_life_rejection_days: int | None = Field(
        None, description="Number of days to reject product before expiration", examples=[20]
    )

    # business related fields
    warehouse_id: str = Field(
        ...,
        max_length=30,
        description="Warehouse identifier",
    )
    owner_id: str = Field(
        ...,
        max_length=30,
        description="Goods owner identifier",
    )

    # physical fields
    length: float | None = Field(None, examples=[12.0], description="Length in centimetres")
    width: float | None = Field(None, examples=[12.0], description="Width in centimetres")
    height: float | None = Field(None, examples=[12.0], description="Height in centimetres")
    volume: float | None = Field(None, examples=[12.0], description="Volume in litres")
    gross_weight: float | None = Field(
        None, examples=[12.0], description="Gross weight in kilograms"
    )
    net_weight: float | None = Field(None, examples=[12.0], description="Net weight in kilograms")

    # price
    currency: str | None = Field(
        None, max_length=3, description="Currency code", examples=["HKD", "CNY"]
    )
    retail_price: float | None = Field(
        None, description="Retail price of the product", examples=[12.00]
    )

    product_group: str | None = Field(
        None, description="Product category or group ID", examples=["12"]
    )

    # Other optional fields
    remarks: str | None = Field(
        None, description="Additional product remarks or notes", examples=["Note", "Remark"]
    )

    class Config:
        arbitrary_types_allowed = True

In [4]:
print(CreateSkuRequest)

<class '__main__.CreateSkuRequest'>


In [5]:
from typing import Any, Type, TypeVar

from langchain_core.documents import Document
from langchain_core.tools import BaseTool
from pydantic import model_validator

In [6]:
R = TypeVar("R", bound=BaseConnexxRequest)


class BaseConnexxRequestTool(BaseTool):
    connexx_request_cls: Type[R]

    @model_validator(mode="before")
    @classmethod
    def init_tool(cls, data) -> Any:
        print(data)
        print(data["connexx_request_cls"].model_fields)
        data["name"] = data["connexx_request_cls"].model_fields["name"].default
        data["description"] = data["connexx_request_cls"].model_fields["description"].default
        data["args_schema"] = data["connexx_request_cls"]
        print(data)
        return data

    def _run(self, *args: Any, **kwargs: Any) -> Any:
        print(f"{self.name} run successfully.")
        try:
            return self.connexx_request_cls(**kwargs).mock()
        except Exception as e:
            return f"{self.name} run failed: {e}"

    @property
    def document(self) -> Document:
        """
        :return: A langchain document to allow a tool to be retrieved.
        """
        return Document(page_content=f"{self.name}- {self.description}")

    @property
    def json_schema(self):
        return self.connexx_request_cls.model_json_schema()

In [7]:
create_sku_tool = BaseConnexxRequestTool(connexx_request_cls=CreateSkuRequest)

{'connexx_request_cls': <class '__main__.CreateSkuRequest'>}
{'name': FieldInfo(annotation=str, required=False, default='insert_sku', metadata=[SkipJsonSchema()]), 'description': FieldInfo(annotation=str, required=False, default='Insert sku', metadata=[SkipJsonSchema()]), 'sku_code': FieldInfo(annotation=str, required=True, description='Product code, item code, or SKU identifier', examples=['10988222'], metadata=[MaxLen(max_length=30)]), 'sku_name': FieldInfo(annotation=str, required=True, description='Product name or description', examples=['HaYa Group YunNan BaiYao 10001'], metadata=[MaxLen(max_length=150)]), 'short_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='Short description or alias for the product', examples=['Yunnan BaiYao'], metadata=[MaxLen(max_length=100)]), 'sku_spec': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='Product specifications', examples=['500mg'], metadata=[MaxLen(max_length=8

In [8]:
create_sku_tool.description

'Insert sku'

In [9]:
create_sku_tool.document

Document(metadata={}, page_content='insert_sku- Insert sku')

In [10]:
create_sku_tool.json_schema

{'properties': {'sku_code': {'description': 'Product code, item code, or SKU identifier',
   'examples': ['10988222'],
   'maxLength': 30,
   'title': 'Sku Code',
   'type': 'string'},
  'sku_name': {'description': 'Product name or description',
   'examples': ['HaYa Group YunNan BaiYao 10001'],
   'maxLength': 150,
   'title': 'Sku Name',
   'type': 'string'},
  'short_name': {'anyOf': [{'maxLength': 100, 'type': 'string'},
    {'type': 'null'}],
   'default': None,
   'description': 'Short description or alias for the product',
   'examples': ['Yunnan BaiYao'],
   'title': 'Short Name'},
  'sku_spec': {'anyOf': [{'maxLength': 80, 'type': 'string'},
    {'type': 'null'}],
   'default': None,
   'description': 'Product specifications',
   'examples': ['500mg'],
   'title': 'Sku Spec'},
  'barcode': {'anyOf': [{'maxLength': 30, 'type': 'string'}, {'type': 'null'}],
   'default': None,
   'description': 'International barcode (EAN/UPC code). Separate by English commas if have more than o

In [11]:
create_sku_tool.invoke(
    {
        "owner_id": "1",
        "warehouse_id": "1",
        "unit": "PC",
        "sku_name": "HaYa Group YunNan BaiYao 10001",
        "sku_code": "10988222",
    }
)

insert_sku run successfully.


"insert_sku run failed: 'CreateSkuRequest' object has no attribute 'mock'"

In [12]:
create_sku_tool.args_schema

__main__.CreateSkuRequest

In [13]:
import math
import re

import numexpr
from langchain_core.tools import tool


def calculator_func(expression: str) -> str:
    """Calculates a math expression using numexpr.

    Useful for when you need to answer questions about math using numexpr.
    This tool is only for math questions and nothing else. Only input
    math expressions.

    Args:
        expression (str): A valid numexpr formatted math expression.

    Returns:
        str: The result of the math expression.
    """

    try:
        local_dict = {"pi": math.pi, "e": math.e}
        output = str(
            numexpr.evaluate(
                expression.strip(),
                global_dict={},  # restrict access to globals
                local_dict=local_dict,  # add common mathematical functions
            )
        )
        return re.sub(r"^\[|\]$", "", output)
    except Exception as e:
        raise ValueError(
            f'calculator("{expression}") raised error: {e}.'
            " Please try again with a valid numerical expression"
        )


calculator: BaseTool = tool(calculator_func)

In [14]:
calculator.name

'calculator_func'

In [15]:
calculator.args

{'expression': {'title': 'Expression', 'type': 'string'}}

In [16]:
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

llm = init_chat_model("azure_openai:gpt-4o", temperature=0.0)

# 定义期望的JSON结构
parser = JsonOutputParser()

prompt = PromptTemplate(
    template="请以JSON格式回答以下问题：{query}\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = prompt | llm | parser

result = chain.invoke({"query": "给我一个笑话"})
print(result)  # 这是一个解析后的JSON对象

{'joke': '为什么计算机喜欢吃零食？因为它们喜欢字节！'}


In [17]:
import json
from functools import partial
from typing import Any, Callable, cast

from langchain_community.llms import OpenAI
from langchain_community.tools.requests.tool import BaseRequestsTool
from langchain_core.prompts import BasePromptTemplate

MAX_RESPONSE_LENGTH = 5000
REQUESTS_GET_TOOL_DESCRIPTION = """Use this to GET content from a website.
Input to the tool should be a json string with 3 keys: "url", "params" and "output_instructions".
The value of "url" should be a string.
The value of "params" should be a dict of the needed and available parameters from the OpenAPI spec related to the endpoint.
If parameters are not needed, or not available, leave it empty.
The value of "output_instructions" should be instructions on what information to extract from the response,
for example the id(s) for a resource(s) that the GET request fetches.
"""
PARSING_GET_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)


def _get_default_llm_chain(prompt: BasePromptTemplate) -> Any:
    from langchain.chains.llm import LLMChain

    return LLMChain(
        llm=OpenAI(),
        prompt=prompt,
    )


def _get_default_llm_chain_factory(
    prompt: BasePromptTemplate,
) -> Callable[[], Any]:
    """Returns a default LLMChain factory."""
    return partial(_get_default_llm_chain, prompt)

In [18]:
import yaml
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

In [19]:
with open("mockserver_openapi.json") as f:
    raw_mockserver_api_spec = json.load(f)
mockserver_api_spec = reduce_openapi_spec(raw_mockserver_api_spec)
mockserver_api_spec

ReducedOpenAPISpec(servers=[{'url': 'http://127.0.0.1:4000'}], description='manager movie records', endpoints=[('GET /movies', None, {'parameters': [], 'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'properties': {'id': {'type': 'integer', 'title': 'Id'}, 'title': {'type': 'string', 'title': 'Title'}, 'year': {'type': 'integer', 'title': 'Year'}}, 'type': 'object', 'required': ['id', 'title', 'year'], 'title': 'Movie'}, 'title': 'Response Get Movies Movies Get'}}}}}), ('POST /movies', None, {'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {'properties': {'id': {'type': 'integer', 'title': 'Id'}, 'title': {'type': 'string', 'title': 'Title'}, 'year': {'type': 'integer', 'title': 'Year'}}, 'type': 'object', 'required': ['id', 'title', 'year'], 'title': 'Movie'}}}}, 'requestBody': {'required': True, 'content': {'application/json': {'schema': {'properties': {'id': {'ty

In [20]:
with open("apisguru_openapi.yaml") as f:
    raw_apisguru_api_spec = yaml.load(f, Loader=yaml.Loader)
apisguru_api_spec = reduce_openapi_spec(raw_apisguru_api_spec)
apisguru_api_spec

ReducedOpenAPISpec(servers=[{'url': 'https://api.apis.guru/v2'}], description='Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\n**Warning**: If you want to be notified about changes in advance please join our [Slack channel](https://join.slack.com/t/mermade/shared_invite/zt-g78g7xir-MLE_CTCcXCdfJfG3CJe9qA).\nClient sample: [[Demo]](https://apis.guru/simple-ui) [[Repo]](https://github.com/APIs-guru/simple-ui)\n', endpoints=[('GET /providers.json', 'List all the providers in the directory\n', {'description': 'List all the providers in the directory\n', 'responses': {'description': 'OK', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'data': {'type': 'array', 'items': {'type': 'string', 'minLength': 1}, 'minItems': 1}}}}}}}), ('GET /{provider}.json', 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n', {'description': 'List all APIs in the directory for a particular

In [21]:
from langchain_community.agent_toolkits.openapi.spec import ReducedOpenAPISpec
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool

In [22]:
API_PLANNER_PROMPT = """You are a planner that plans a sequence of API calls to assist with user queries against an API.

You should:
1) evaluate whether the user query can be solved by the API documented below. If no, say why.
2) if yes, generate a plan of API calls and say what they are doing step by step.
3) If the plan includes a DELETE call, you should always return an ask from the User for authorization first unless the User has specifically asked to delete something.

You should only use API endpoints documented below ("Endpoints you can use:").
You can only use the DELETE tool if the User has specifically asked to delete something. Otherwise, you should return a request authorization from the User first.
Some user queries can be resolved in a single API call, but some will require several API calls.
The plan will be passed to an API controller that can format it into web requests and return the responses.

----

Here are some examples:

Fake endpoints for examples:
GET /user to get information about the current user
GET /products/search search across products
POST /users/{{id}}/cart to add products to a user's cart
PATCH /users/{{id}}/cart to update a user's cart
PUT /users/{{id}}/coupon to apply idempotent coupon to a user's cart
DELETE /users/{{id}}/cart to delete a user's cart

User query: tell me a joke
Plan: Sorry, this API's domain is shopping, not comedy.

User query: I want to buy a couch
Plan: 1. GET /products with a query param to search for couches
2. GET /user to find the user's id
3. POST /users/{{id}}/cart to add a couch to the user's cart

User query: I want to add a lamp to my cart
Plan: 1. GET /products with a query param to search for lamps
2. GET /user to find the user's id
3. PATCH /users/{{id}}/cart to add a lamp to the user's cart

User query: I want to add a coupon to my cart
Plan: 1. GET /user to find the user's id
2. PUT /users/{{id}}/coupon to apply the coupon

User query: I want to delete my cart
Plan: 1. GET /user to find the user's id
2. DELETE required. Did user specify DELETE or previously authorize? Yes, proceed.
3. DELETE /users/{{id}}/cart to delete the user's cart

User query: I want to start a new cart
Plan: 1. GET /user to find the user's id
2. DELETE required. Did user specify DELETE or previously authorize? No, ask for authorization.
3. Are you sure you want to delete your cart?
----

Here are endpoints you can use. Do not reference any of the endpoints above.

{endpoints}

----

User query: {query}
Plan:"""
API_PLANNER_TOOL_NAME = "api_planner"
API_PLANNER_TOOL_DESCRIPTION = f"Can be used to generate the right API calls to assist with a user query, like {API_PLANNER_TOOL_NAME}(query). Should always be called before trying to call the API controller."

In [23]:
def _create_api_planner_tool(api_spec: ReducedOpenAPISpec, llm: BaseLanguageModel) -> Tool:
    from langchain.chains.llm import LLMChain

    endpoint_descriptions = [f"{name} {description}" for name, description, _ in api_spec.endpoints]
    prompt = PromptTemplate(
        template=API_PLANNER_PROMPT,
        input_variables=["query"],
        partial_variables={"endpoints": "- " + "- ".join(endpoint_descriptions)},
    )
    print("- " + "- ".join(endpoint_descriptions))
    chain = LLMChain(llm=llm, prompt=prompt)
    tool = Tool(
        name=API_PLANNER_TOOL_NAME,
        description=API_PLANNER_TOOL_DESCRIPTION,
        func=chain.run,
    )
    return tool

In [24]:
api_planner_tool = _create_api_planner_tool(apisguru_api_spec, llm)

- GET /providers.json List all the providers in the directory
- GET /{provider}.json List all APIs in the directory for a particular providerName
Returns links to the individual API entry for each API.
- GET /{provider}/services.json List all serviceNames in the directory for a particular providerName
- GET /specs/{provider}/{api}.json Returns the API entry for one specific version of an API where there is no serviceName.- GET /specs/{provider}/{service}/{api}.json Returns the API entry for one specific version of an API where there is a serviceName.- GET /list.json List all APIs in the directory.
Returns links to the OpenAPI definitions for each API in the directory.
If API exist in multiple versions `preferred` one is explicitly marked.
Some basic info from the OpenAPI definition is cached inside each object.
This allows you to generate some simple views without needing to fetch the OpenAPI definition for each API.
- GET /metrics.json Some basic metrics for the entire directory.
Just

C:\Users\Admin\AppData\Local\Temp\ipykernel_36752\3324045210.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [25]:
api_planner_tool.invoke("List all the providers in the directory")

'1. GET /providers.json to list all the providers in the directory'

In [26]:
class RequestsGetToolWithParsing(BaseRequestsTool, BaseTool):
    """Requests GET tool with LLM-instructed extraction of truncated responses."""

    name: str = "requests_get"
    """Tool name."""
    description: str = REQUESTS_GET_TOOL_DESCRIPTION
    """Tool description."""
    response_length: int = MAX_RESPONSE_LENGTH
    """Maximum length of the response to be returned."""
    llm_chain: Any = Field(default_factory=_get_default_llm_chain_factory(PARSING_GET_PROMPT))
    """LLMChain used to extract the response."""

    def _run(self, text: str) -> str:
        from langchain.output_parsers.json import parse_json_markdown

        try:
            data = parse_json_markdown(text)
        except json.JSONDecodeError as e:
            raise e
        data_params = data.get("params")
        response: str = cast(str, self.requests_wrapper.get(data["url"], params=data_params))
        # response = response[: self.response_length]
        return self.llm_chain.predict(
            response=response, instructions=data["output_instructions"]
        ).strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()

In [27]:
from langchain.chains.llm import LLMChain
from langchain_community.utilities.requests import RequestsWrapper

requests_wrapper = RequestsWrapper()
get_llm_chain = LLMChain(llm=llm, prompt=PARSING_GET_PROMPT)
api_controller_tool = RequestsGetToolWithParsing(
    requests_wrapper=requests_wrapper,
    llm_chain=get_llm_chain,
    allow_dangerous_requests=True,
)

result = api_controller_tool.invoke(
    {
        "text": """```json
{
  "url": "https://api.apis.guru/v2/providers.json",
  "params": {},
  "output_instructions": "Extract the list of provider names."
}
```"""
    }
)
print(result)

1forge  
1password  
6-dot-authentiqio  
ably  
abstractapi  
adafruit  
adobe  
adyen  
afterbanks  
agco-ats  
aiception  
airbyte  
airport-web  
akeneo  
alertersystem  
amadeus  
amazonaws  
amentum  
anchore  
apache  
apacta  
api.ebay  
api.gov  
api.video  
api2cart  
api2pdf  
apicurio  
apidapp  
apideck  
apigee  
apimatic  
apis.guru  
apisetu  
apispot  
apiz.ebay  
appcenter  
apple  
apptigent  
appveyor  
appwrite  
archive  
arespass  
art19  
asana  
asuarez  
atlassian  
ato  
aucklandmuseum  
authentiq  
autodealerdata  
autotask  
avaza  
aviationdata  
axesso  
azure  
balldontlie  
bandsintown  
bbc  
bbci  
bclaws  
beanstream  
beezup  
betfair  
bethmardutho  
bhagavadgita  
biapi  
bigdatacloud  
bigoven  
bigredcloud  
bikewise  
billbee  
billingo  
bintable  
bitbucket  
biztoc  
blazemeter  
bluemix  
botify  
botschaft  
box  
brainbi  
brandlovers  
braze  
brex  
bridgedb  
britbox  
browshot  
bufferapp  
bulksms  
bungie  
bunq  
byautomata  
c19qrs

In [28]:
import keyword
import re


def make_valid_identifier(s: str, prefix: str = "_", fallback: str = "_") -> str:
    """
    将任意字符串转换成合法的 Python 标识符。
    - s: 原始字符串
    - prefix: 若首字符不合法，使用的前缀
    - fallback: 若结果为空时的默认名称
    """
    # 替换不合法字符为下划线
    cleaned = re.sub(r"[^0-9a-zA-Z_]", "_", s.strip())
    # 若首字符为数字，加前缀
    if cleaned and cleaned[0].isdigit():
        cleaned = prefix + cleaned
    # 若清洗后为空或全是下划线，用 fallback
    if not cleaned or cleaned.strip("_") == "":
        cleaned = fallback
    # 若和 Python 关键字冲突，加后缀避免冲突
    if keyword.iskeyword(cleaned):
        cleaned += "_"
    return cleaned


In [29]:
make_valid_identifier(apisguru_api_spec.endpoints[1][0])

'GET___provider__json'

In [30]:
apisguru_api_spec.endpoints[1][1]

'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n'

In [31]:
schema = {
    "type": "object",
    "properties": {"text": apisguru_api_spec.endpoints[1][2]["parameters"][0]["schema"]},
}
schema

{'type': 'object',
 'properties': {'text': {'type': 'string',
   'minLength': 1,
   'maxLength': 255,
   'example': 'apis.guru'}}}

In [32]:
from json_schema_to_pydantic import create_model

testClass = create_model(schema)
a = testClass()
a

DynamicModel(text=None)

In [33]:
class RequestsGetToolWithParsingCustom(BaseRequestsTool, BaseTool):
    """Requests GET tool with LLM-instructed extraction of truncated responses."""

    name: str = "requests_get"
    """Tool name."""
    description: str = REQUESTS_GET_TOOL_DESCRIPTION
    """Tool description."""
    response_length: int = MAX_RESPONSE_LENGTH
    """Maximum length of the response to be returned."""
    llm_chain: Any = Field(default_factory=_get_default_llm_chain_factory(PARSING_GET_PROMPT))
    """LLMChain used to extract the response."""

    def _run(self, text: str) -> str:
        # from langchain.output_parsers.json import parse_json_markdown

        # try:
        #     data = parse_json_markdown(text)
        # except json.JSONDecodeError as e:
        #     raise e
        # data_params = data.get("params")
        data_params = text
        url = f"https://api.apis.guru/v2/{text}.json"
        response: str = cast(str, self.requests_wrapper.get(url, params={}))
        # response = response[: self.response_length]
        return self.llm_chain.predict(response=response, instructions="show the result").strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()

In [ ]:
api_controller_tool_custom = RequestsGetToolWithParsingCustom(
    name=make_valid_identifier(apisguru_api_spec.endpoints[1][0]),
    description=apisguru_api_spec.endpoints[1][1],
    args_schema=create_model(schema),
    requests_wrapper=requests_wrapper,
    llm_chain=get_llm_chain,
    allow_dangerous_requests=True,
)

In [35]:
api_controller_tool_custom.name

'GET___provider__json'

In [36]:
from langgraph.prebuilt import create_react_agent

system_prompt = """You are an agent that assists with user queries against API, things like querying information or creating resources.
Some user queries can be resolved in a single API call, particularly if you can find appropriate params from the OpenAPI spec; though some require several API calls.
You should always plan your API calls first, and then execute the plan second.
If the plan includes a DELETE call, be sure to ask the User for authorization first unless the User has specifically asked to delete something.
You should never return information without executing the api_controller tool.

Don't forget to add base url: {api_url} to the api_controller tool.
""".format(api_url="https://api.apis.guru/v2/")

agent = create_react_agent(
    llm,
    [api_planner_tool, api_controller_tool_custom],
    prompt=system_prompt,
)

In [37]:
# question = "List all the providers in the directory"
question = "List all APIs in the directory for apis.guru"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

List all APIs in the directory for apis.guru
================================== Ai Message ==================================
Tool Calls:
  api_planner (call_dZUSDfxkuw4Zggibcs4NPiWn)
 Call ID: call_dZUSDfxkuw4Zggibcs4NPiWn
  Args:
    __arg1: List all APIs in the directory for apis.guru
================================= Tool Message =================================
Name: api_planner

1. GET /apis.guru.json to list all APIs in the directory for the provider "apis.guru".
================================== Ai Message ==================================
Tool Calls:
  GET___provider__json (call_aUmTjtWxCxGUsDeQvnSEFbyQ)
 Call ID: call_aUmTjtWxCxGUsDeQvnSEFbyQ
  Args:
================================= Tool Message =================================
Name: GET___provider__json

Error: TypeError("RequestsGetToolWithParsingCustom._run() missing 1 required positional argument: 'text'")
 Please fix your mistakes.
===